# Proof of concept of working of AutoEncoder

In [3]:
import torch


def AutoEncoderLoss(output, target):
    # MSE loss
    loss = torch.mean((output - target)**2)
    return loss


class AutoEncoder(torch.nn.Module):

    def __init__(self, D_in, H, shared_decoder):
        # Encoder
        super(AutoEncoder, self).__init__()
        self.w1 = torch.nn.Linear(D_in, H)
        self.w2 = torch.nn.Linear(H, H)
        
        self.activation = torch.nn.functional.relu
        
        # Decoder
        self.decoder = shared_decoder
        
    def forward(self, x):
        # Use encoder for encoding
        x = self.w1(x)
        x = self.activation(x)
        x = self.w2(x)
        
        # Decoder
        x = self.decoder(x)
        
        return x

In [3]:
# Dummy test for AutoEncoder
from model import gan

# Network param settings
embedding_dim = 8
internal_dim = 300
output_dim = 1

# Train hyperparameters
epochs = 100
languages = {'src': ['de', 'nl'], 'trgt': ['en']}


# Architectures
net = gan.GAN(embedding_dim, internal_dim, output_dim, languages['src'])

ae = AutoEncoder(embedding_dim, internal_dim, net.generator.decoder)
optimizer_ae = torch.optim.Adam(ae.parameters(),
                                         lr=0.01, betas=(0.9, 0.999), eps=1e-08, 
                                         weight_decay=0., amsgrad=False)

# Create toy train dataset
rows, cols = 10, embedding_dim
train_data = torch.rand(rows, cols)

# Report outcome before training
reproduction = ae(train_data)
print('Input data:\n', train_data)
print('Reproduced data:\n', reproduction)
print('MSE loss on reproducing input:', AutoEncoderLoss(reproduction, train_data))
print('Start training.')
# Train
for i in range(1000):
    ae.train()
    optimizer_ae.zero_grad()
    y_pred = ae(train_data)
    loss = AutoEncoderLoss(y_pred, train_data)
    loss.backward()
    optimizer_ae.step()
    print('Epoch', i, 'Loss:', loss.detach().numpy())
    
# Report outcome after training
reproduction = ae(train_data)
print('Training finished.')
print('Input data:\n', train_data)
print('Reproduced data:\n', reproduction)
print('MSE loss on reproducing input:', AutoEncoderLoss(reproduction, train_data))

NameError: name 'AutoEncoder' is not defined

# Proof of concept of working of NoisyAutoEncoder

In [2]:
import torch


def AutoEncoderLoss(output, target):
    # MSE loss
    loss = torch.mean((output - target)**2)
    return loss


def bound_noise(shape, min_=-0.05, max_=0.05):
    return (min_ - max_) * torch.rand(shape) + max_



class NoisyAutoEncoder(torch.nn.Module):

    def __init__(self, shared_encoder, D_in, H, D_out):
        super(NoisyAutoEncoder, self).__init__()
        # Encoder
        self.encoder = shared_encoder
        
        # Decoder
        self.w1 = torch.nn.Linear(D_in, H)
        self.w2 = torch.nn.Linear(H, H)
        
        self.activation = torch.nn.functional.relu
        
        self.noise = bound_noise
        
    def forward(self, x):
        # Use encoder for encoding, but reproduce original encoder input in the presence of noise
        # Encoder
        x = self.encoder(x)
        
        # Noisy Decoder
        x = x + self.noise(x.shape)
        x = self.w1(x)
        x = x + self.noise(x.shape)
        x = self.activation(x)
        x = self.w2(x)
        
        return x

In [8]:
# Dummy test for NoisyAutoEncoder
from model import gan

# Network param settings
embedding_dim = 8
internal_dim = 8
output_dim = 1

# Train hyperparameters
epochs = 100
languages = {'src': ['de', 'nl'], 'trgt': ['en']}


# Architectures
net = gan.GAN(embedding_dim, internal_dim, output_dim, languages['src'])

nae = NoisyAutoEncoder(net.generator.encoders['de'], embedding_dim, internal_dim, embedding_dim)
optimizer_nae = torch.optim.Adam(nae.parameters(),
                                         lr=0.01, betas=(0.9, 0.999), eps=1e-08, 
                                         weight_decay=0., amsgrad=False)

# Create toy train dataset
rows, cols = 10, embedding_dim
train_data = torch.rand(rows, cols)

# Report outcome before training
reproduction = nae(train_data)
print('Input data:\n', train_data)
print('Reproduced data:\n', reproduction)
print('MSE loss on reproducing input:', AutoEncoderLoss(reproduction, train_data))
print('Start training.')
# Train
for i in range(1000):
    nae.train()
    optimizer_nae.zero_grad()
    y_pred = nae(train_data)
    loss = AutoEncoderLoss(y_pred, train_data)
    loss.backward()
    optimizer_nae.step()
    print('Epoch', i, 'Loss:', loss.detach().numpy())
    
# Report outcome after training
reproduction = nae(train_data)
print('Training finished.')
print('Input data:\n', train_data)
print('Reproduced data:\n', reproduction)
print('MSE loss on reproducing input:', AutoEncoderLoss(reproduction, train_data))

Input data:
 tensor([[0.7925, 0.1257, 0.1079, 0.2810, 0.7521, 0.8251, 0.8766, 0.4995],
        [0.9899, 0.9532, 0.0633, 0.4915, 0.3611, 0.1932, 0.4105, 0.2878],
        [0.7229, 0.7977, 0.7767, 0.6142, 0.7200, 0.0512, 0.8869, 0.6952],
        [0.7129, 0.6997, 0.3178, 0.8022, 0.3644, 0.0837, 0.7434, 0.6590],
        [0.6267, 0.7003, 0.2751, 0.1730, 0.7817, 0.5792, 0.9970, 0.9297],
        [0.2054, 0.0021, 0.5645, 0.8659, 0.7482, 0.0415, 0.2398, 0.6980],
        [0.3510, 0.8873, 0.3284, 0.8887, 0.1090, 0.8524, 0.1422, 0.7692],
        [0.6656, 0.8468, 0.4786, 0.2145, 0.1358, 0.7119, 0.5921, 0.6690],
        [0.3217, 0.7834, 0.9507, 0.1533, 0.4751, 0.8010, 0.9633, 0.4481],
        [0.6641, 0.7940, 0.5648, 0.8635, 0.5375, 0.0980, 0.1027, 0.7972]])
Reproduced data:
 tensor([[ 0.1210,  0.0732, -0.0659, -0.3304,  0.1877, -0.2695, -0.2089,  0.0197],
        [ 0.1274,  0.0638, -0.0724, -0.3254,  0.1948, -0.3065, -0.1990,  0.0265],
        [ 0.1461,  0.0522, -0.0892, -0.3332,  0.1742, -0.3144, -

Epoch 276 Loss: 0.038245767
Epoch 277 Loss: 0.037742153
Epoch 278 Loss: 0.037752338
Epoch 279 Loss: 0.03807326
Epoch 280 Loss: 0.037419386
Epoch 281 Loss: 0.036723696
Epoch 282 Loss: 0.03705345
Epoch 283 Loss: 0.036898088
Epoch 284 Loss: 0.035912313
Epoch 285 Loss: 0.036062002
Epoch 286 Loss: 0.036112614
Epoch 287 Loss: 0.035649143
Epoch 288 Loss: 0.035568856
Epoch 289 Loss: 0.03536073
Epoch 290 Loss: 0.035278153
Epoch 291 Loss: 0.035323933
Epoch 292 Loss: 0.034944408
Epoch 293 Loss: 0.03476105
Epoch 294 Loss: 0.03410182
Epoch 295 Loss: 0.034375884
Epoch 296 Loss: 0.03399612
Epoch 297 Loss: 0.03374473
Epoch 298 Loss: 0.033575807
Epoch 299 Loss: 0.033737857
Epoch 300 Loss: 0.033542514
Epoch 301 Loss: 0.033203643
Epoch 302 Loss: 0.032669757
Epoch 303 Loss: 0.033388756
Epoch 304 Loss: 0.03247658
Epoch 305 Loss: 0.032601915
Epoch 306 Loss: 0.032435186
Epoch 307 Loss: 0.032186747
Epoch 308 Loss: 0.032274198
Epoch 309 Loss: 0.032169513
Epoch 310 Loss: 0.031597096
Epoch 311 Loss: 0.031628273


Epoch 683 Loss: 0.022082498
Epoch 684 Loss: 0.022143563
Epoch 685 Loss: 0.022148028
Epoch 686 Loss: 0.021736728
Epoch 687 Loss: 0.02189279
Epoch 688 Loss: 0.022250507
Epoch 689 Loss: 0.021817995
Epoch 690 Loss: 0.021938853
Epoch 691 Loss: 0.022008736
Epoch 692 Loss: 0.021911476
Epoch 693 Loss: 0.021918477
Epoch 694 Loss: 0.021762671
Epoch 695 Loss: 0.021927653
Epoch 696 Loss: 0.022167938
Epoch 697 Loss: 0.021914113
Epoch 698 Loss: 0.021859754
Epoch 699 Loss: 0.02172016
Epoch 700 Loss: 0.02190641
Epoch 701 Loss: 0.021895258
Epoch 702 Loss: 0.021662574
Epoch 703 Loss: 0.021923695
Epoch 704 Loss: 0.021809485
Epoch 705 Loss: 0.021932177
Epoch 706 Loss: 0.021887224
Epoch 707 Loss: 0.021722246
Epoch 708 Loss: 0.021703405
Epoch 709 Loss: 0.02209816
Epoch 710 Loss: 0.021806529
Epoch 711 Loss: 0.021687105
Epoch 712 Loss: 0.021865409
Epoch 713 Loss: 0.022150282
Epoch 714 Loss: 0.02196797
Epoch 715 Loss: 0.021753564
Epoch 716 Loss: 0.021843122
Epoch 717 Loss: 0.022031264
Epoch 718 Loss: 0.0219832

# Proof of concept of working of linear-GAN

# Proof of concept of working of non-linear-GAN

In [4]:
# Dummy test for GAN
import torch
from model import gan


def AutoEncoderLoss(output, target):
    # MSE loss
    loss = torch.mean((output - target)**2)
    return loss


# Network param settings
embedding_dim = 8
internal_dim = 300
output_dim = 1

# Train hyperparameters
epochs = 100
languages = {'src': ['de'], 'trgt': ['en']}
nr_langs = 2
batch_size = 10
real_label = 1


# Architecture
net = gan.GAN(embedding_dim, internal_dim, output_dim, languages['src'])

# Get optimizers; 1 per source language of encoder and 1 for discriminator
optimizers = {'gen': {}}
for lang in languages['src']:
    optimizers['gen'][lang] = torch.optim.Adam([{'params': net.generator.encoders[lang].parameters()},
                                                {'params': net.generator.decoder.parameters()}
                                               ],
                                               lr=0.001, betas=(0.9, 0.999), eps=1e-08, 
                                               weight_decay=0.01, amsgrad=False)
optimizers['dis'] = torch.optim.Adam(net.discriminator.parameters(),
                                     lr=0.001, betas=(0.9, 0.999), eps=1e-08, 
                                     weight_decay=0.001, amsgrad=False)

# Loss function
criterion = torch.nn.BCELoss()


# Create toy train dataset
rows, cols = batch_size, embedding_dim
train_data = torch.rand(rows, cols)


# Report outcome before training
reproduction = net.generator(train_data, 'de')
print('Input data:\n', train_data)
print('Reproduced data:\n', reproduction)
print('MSE loss on reproducing input:', AutoEncoderLoss(reproduction, train_data))
print('Start training.')


# Train
for epoch in range(10000):

    # Update discriminator #
    
    net.discriminator.train()
    net.generator.eval()
    net.discriminator.zero_grad()

    # Labels
    y_true = torch.zeros([nr_langs*batch_size])
    y_true[0:batch_size] = real_label  # First elements are target embeddings


    # Predictions all-real minibatch
    y_preds = torch.zeros([nr_langs*batch_size, 1]) 
    y_preds[0:batch_size] = net.discriminator(train_data)

    # Predictions all-fake minibatch
    x_trans = net.generator(train_data, 'de')  # Generate fake data aka translate
    y_preds[batch_size:] = net.discriminator(x_trans)
    loss_dis = criterion(y_preds, y_true)
    loss_dis.backward()    # Compute gradients only for discriminator
    optimizers['dis'].step() 
    
    # Update generator #
    
    net.generator.train()
    net.discriminator.eval()
    net.generator.zero_grad()
    
    # Labels
    y_true = torch.full((batch_size,), real_label)
    
    # Predictions all-fake minibatch
    x_trans = net.generator(train_data, 'de')
    y_pred = net.discriminator(x_trans)
    loss_gen = criterion(y_pred, y_true)
    loss_gen.backward()    # Compute gradients only for generator
    optimizers['gen']['de'].step()

    print('Epoch', epoch, 'Generator loss:', loss_gen.detach().numpy(), 
          'Discriminator loss:', loss_dis.detach().numpy())
    
    
# Report outcome after training
reproduction = net.generator(train_data, 'de')
print('Training finished.')
print('Input data:\n', train_data)
print('Reproduced data:\n', reproduction)
print('MSE loss on reproducing input:', AutoEncoderLoss(reproduction, train_data))

Input data:
 tensor([[0.3262, 0.3874, 0.9251, 0.9934, 0.6071, 0.8287, 0.4139, 0.2610],
        [0.2380, 0.4215, 0.3736, 0.2866, 0.7767, 0.9396, 0.7336, 0.3588],
        [0.2373, 0.5886, 0.3977, 0.5954, 0.6882, 0.9674, 0.7818, 0.3925],
        [0.8115, 0.5452, 0.9320, 0.2296, 0.3907, 0.8335, 0.2031, 0.6049],
        [0.3788, 0.5697, 0.4299, 0.4714, 0.9619, 0.8276, 0.6430, 0.5305],
        [0.4828, 0.1298, 0.5827, 0.3718, 0.0118, 0.3598, 0.8698, 0.3966],
        [0.7637, 0.3759, 0.6307, 0.4375, 0.8698, 0.5631, 0.7726, 0.0767],
        [0.6337, 0.7312, 0.2318, 0.1262, 0.5005, 0.8179, 0.9828, 0.3621],
        [0.5863, 0.8876, 0.6611, 0.0289, 0.0325, 0.6636, 0.4850, 0.3284],
        [0.2700, 0.8029, 0.4334, 0.5841, 0.5684, 0.8943, 0.2297, 0.3848]])
Reproduced data:
 tensor([[-0.0495, -0.0459,  0.0039, -0.0099, -0.0937, -0.0235, -0.0085,  0.0926],
        [-0.0465, -0.0260, -0.0129, -0.0011, -0.0874, -0.0058,  0.0174,  0.0681],
        [-0.0590, -0.0258, -0.0221, -0.0060, -0.0849, -0.0078,  

/usr/local/lib/python3.5/dist-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
/pytorch/aten/src/ATen/native/TensorFactories.cpp:361: UserWarning: Deprecation warning: In a future PyTorch release torch.full will no longer return tensors of floating dtype by default. Instead, a bool fill_value will return a tensor of torch.bool dtype, and an integral fill_value will return a tensor of torch.long dtype. Set the optional `dtype` or `out` arguments to suppress this warning.
/usr/local/lib/python3.5/dist-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target

Epoch 30 Generator loss: 1.0406917 Discriminator loss: 0.7972643
Epoch 31 Generator loss: 0.994096 Discriminator loss: 0.7511407
Epoch 32 Generator loss: 0.95310307 Discriminator loss: 0.7108614
Epoch 33 Generator loss: 0.93349874 Discriminator loss: 0.6732998
Epoch 34 Generator loss: 0.92253506 Discriminator loss: 0.64258176
Epoch 35 Generator loss: 0.91726446 Discriminator loss: 0.6201581
Epoch 36 Generator loss: 0.9314596 Discriminator loss: 0.6007957
Epoch 37 Generator loss: 0.9588574 Discriminator loss: 0.58492255
Epoch 38 Generator loss: 0.9764387 Discriminator loss: 0.5787733
Epoch 39 Generator loss: 0.9630799 Discriminator loss: 0.5866941
Epoch 40 Generator loss: 0.89899236 Discriminator loss: 0.6192489
Epoch 41 Generator loss: 0.8167303 Discriminator loss: 0.66918784
Epoch 42 Generator loss: 0.73013604 Discriminator loss: 0.73388565
Epoch 43 Generator loss: 0.6407397 Discriminator loss: 0.8148195
Epoch 44 Generator loss: 0.6030246 Discriminator loss: 0.8758815
Epoch 45 Generat

Epoch 156 Generator loss: 0.64293253 Discriminator loss: 0.7556182
Epoch 157 Generator loss: 0.6449634 Discriminator loss: 0.76130074
Epoch 158 Generator loss: 0.65261173 Discriminator loss: 0.7649275
Epoch 159 Generator loss: 0.66673684 Discriminator loss: 0.76564825
Epoch 160 Generator loss: 0.693386 Discriminator loss: 0.76104677
Epoch 161 Generator loss: 0.7307733 Discriminator loss: 0.75128365
Epoch 162 Generator loss: 0.77077615 Discriminator loss: 0.74002063
Epoch 163 Generator loss: 0.80915964 Discriminator loss: 0.7292516
Epoch 164 Generator loss: 0.8422743 Discriminator loss: 0.719576
Epoch 165 Generator loss: 0.86581886 Discriminator loss: 0.7120177
Epoch 166 Generator loss: 0.87612647 Discriminator loss: 0.70746815
Epoch 167 Generator loss: 0.87159747 Discriminator loss: 0.70655614
Epoch 168 Generator loss: 0.85002536 Discriminator loss: 0.7101574
Epoch 169 Generator loss: 0.8176646 Discriminator loss: 0.7168616
Epoch 170 Generator loss: 0.77722037 Discriminator loss: 0.725

Epoch 308 Generator loss: 0.45864677 Discriminator loss: 0.769395
Epoch 309 Generator loss: 0.41574144 Discriminator loss: 0.8111027
Epoch 310 Generator loss: 0.38259757 Discriminator loss: 0.8523485
Epoch 311 Generator loss: 0.3625489 Discriminator loss: 0.887613
Epoch 312 Generator loss: 0.35961133 Discriminator loss: 0.9088313
Epoch 313 Generator loss: 0.3777057 Discriminator loss: 0.9117306
Epoch 314 Generator loss: 0.41816315 Discriminator loss: 0.89758074
Epoch 315 Generator loss: 0.48293638 Discriminator loss: 0.86885816
Epoch 316 Generator loss: 0.5747997 Discriminator loss: 0.8301343
Epoch 317 Generator loss: 0.692929 Discriminator loss: 0.7887856
Epoch 318 Generator loss: 0.8305257 Discriminator loss: 0.75114334
Epoch 319 Generator loss: 0.9723752 Discriminator loss: 0.72237885
Epoch 320 Generator loss: 1.1019495 Discriminator loss: 0.70448905
Epoch 321 Generator loss: 1.1954831 Discriminator loss: 0.69754136
Epoch 322 Generator loss: 1.2363262 Discriminator loss: 0.70001924


...

Epoch 9456 Generator loss: 0.6768762 Discriminator loss: 0.6778804
Epoch 9457 Generator loss: 0.678047 Discriminator loss: 0.6821221
Epoch 9458 Generator loss: 0.66980726 Discriminator loss: 0.6945256
Epoch 9459 Generator loss: 0.67230666 Discriminator loss: 0.70434433
Epoch 9460 Generator loss: 0.67447233 Discriminator loss: 0.71589947
Epoch 9461 Generator loss: 0.63271123 Discriminator loss: 0.74675196
Epoch 9462 Generator loss: 0.68934065 Discriminator loss: 0.72486955
Epoch 9463 Generator loss: 0.7330898 Discriminator loss: 0.7189442
Epoch 9464 Generator loss: 0.7816485 Discriminator loss: 0.6945194
Epoch 9465 Generator loss: 0.8289444 Discriminator loss: 0.68048614
Epoch 9466 Generator loss: 0.84882724 Discriminator loss: 0.66791236
Epoch 9467 Generator loss: 0.847658 Discriminator loss: 0.66604424
Epoch 9468 Generator loss: 0.8249302 Discriminator loss: 0.6731056
Epoch 9469 Generator loss: 0.82720196 Discriminator loss: 0.66732776
Epoch 9470 Generator loss: 0.8033867 Discriminato

Epoch 9592 Generator loss: 0.8502469 Discriminator loss: 0.7099492
Epoch 9593 Generator loss: 0.84744084 Discriminator loss: 0.71897674
Epoch 9594 Generator loss: 0.8732556 Discriminator loss: 0.7115881
Epoch 9595 Generator loss: 0.8619524 Discriminator loss: 0.7066985
Epoch 9596 Generator loss: 0.8356608 Discriminator loss: 0.7106668
Epoch 9597 Generator loss: 0.8882438 Discriminator loss: 0.68667924
Epoch 9598 Generator loss: 0.883037 Discriminator loss: 0.67885524
Epoch 9599 Generator loss: 0.7928682 Discriminator loss: 0.6993031
Epoch 9600 Generator loss: 0.8826748 Discriminator loss: 0.65018934
Epoch 9601 Generator loss: 0.87414086 Discriminator loss: 0.64006025
Epoch 9602 Generator loss: 0.7810816 Discriminator loss: 0.6634618
Epoch 9603 Generator loss: 0.7695555 Discriminator loss: 0.6595925
Epoch 9604 Generator loss: 0.7916068 Discriminator loss: 0.6409988
Epoch 9605 Generator loss: 0.74585813 Discriminator loss: 0.6531631
Epoch 9606 Generator loss: 0.70471954 Discriminator los

Epoch 9722 Generator loss: 0.7802344 Discriminator loss: 0.6231138
Epoch 9723 Generator loss: 0.8305315 Discriminator loss: 0.60062706
Epoch 9724 Generator loss: 0.79464775 Discriminator loss: 0.6130341
Epoch 9725 Generator loss: 0.72911686 Discriminator loss: 0.6383932
Epoch 9726 Generator loss: 0.8146416 Discriminator loss: 0.5946628
Epoch 9727 Generator loss: 0.82138026 Discriminator loss: 0.5874782
Epoch 9728 Generator loss: 0.78705305 Discriminator loss: 0.59891546
Epoch 9729 Generator loss: 0.6915435 Discriminator loss: 0.64371234
Epoch 9730 Generator loss: 0.593461 Discriminator loss: 0.7113004
Epoch 9731 Generator loss: 0.40996504 Discriminator loss: 0.88615453
Epoch 9732 Generator loss: 0.3690324 Discriminator loss: 0.97326374
Epoch 9733 Generator loss: 0.4360413 Discriminator loss: 0.95204735
Epoch 9734 Generator loss: 0.6265382 Discriminator loss: 0.86348915
Epoch 9735 Generator loss: 1.0289689 Discriminator loss: 0.712396
Epoch 9736 Generator loss: 1.5240487 Discriminator l

Epoch 9845 Generator loss: 0.70049614 Discriminator loss: 0.6234869
Epoch 9846 Generator loss: 0.6632957 Discriminator loss: 0.64202374
Epoch 9847 Generator loss: 0.60083973 Discriminator loss: 0.6800369
Epoch 9848 Generator loss: 0.6209415 Discriminator loss: 0.6739998
Epoch 9849 Generator loss: 0.6284403 Discriminator loss: 0.6778884
Epoch 9850 Generator loss: 0.57414 Discriminator loss: 0.7178779
Epoch 9851 Generator loss: 0.58528525 Discriminator loss: 0.71397126
Epoch 9852 Generator loss: 0.6183527 Discriminator loss: 0.69952375
Epoch 9853 Generator loss: 0.58842665 Discriminator loss: 0.73344195
Epoch 9854 Generator loss: 0.6286393 Discriminator loss: 0.7226342
Epoch 9855 Generator loss: 0.645082 Discriminator loss: 0.72512245
Epoch 9856 Generator loss: 0.62840885 Discriminator loss: 0.74230164
Epoch 9857 Generator loss: 0.74641716 Discriminator loss: 0.68289
Epoch 9858 Generator loss: 0.68987805 Discriminator loss: 0.72256136
Epoch 9859 Generator loss: 0.7725724 Discriminator lo

Epoch 9969 Generator loss: 0.7691988 Discriminator loss: 0.62063843
Epoch 9970 Generator loss: 0.7725897 Discriminator loss: 0.6179931
Epoch 9971 Generator loss: 0.77691305 Discriminator loss: 0.6157187
Epoch 9972 Generator loss: 0.78298324 Discriminator loss: 0.6135236
Epoch 9973 Generator loss: 0.78958315 Discriminator loss: 0.6109382
Epoch 9974 Generator loss: 0.7947884 Discriminator loss: 0.6082574
Epoch 9975 Generator loss: 0.79956186 Discriminator loss: 0.6054525
Epoch 9976 Generator loss: 0.80287683 Discriminator loss: 0.6021556
Epoch 9977 Generator loss: 0.8056346 Discriminator loss: 0.59913206
Epoch 9978 Generator loss: 0.8087987 Discriminator loss: 0.5957719
Epoch 9979 Generator loss: 0.8102366 Discriminator loss: 0.59161717
Epoch 9980 Generator loss: 0.81359375 Discriminator loss: 0.5887726
Epoch 9981 Generator loss: 0.8186854 Discriminator loss: 0.58540946
Epoch 9982 Generator loss: 0.82484865 Discriminator loss: 0.5816591
Epoch 9983 Generator loss: 0.8310461 Discriminator 

In [ ]:
# Proof of concept of working of non-linear-GAN with AutoEncoder and NoisyAutoEncoder

In [13]:
# Dummy test for GAN
import torch
from model import gan

# Loss functions
criterion = torch.nn.BCELoss()

def AutoEncoderLoss(output, target):
    # MSE loss
    loss = torch.mean((output - target)**2)
    return loss


# Network param settings
embedding_dim = 8
internal_dim = 8
output_dim = 1

# Train hyperparameters
epochs = 100
languages = {'src': ['de'], 'trgt': ['en']}
nr_langs = 2
batch_size = 10
real_label = 1


# Architectures
### GAN ###
net = gan.GAN(embedding_dim, internal_dim, output_dim, languages['src'])
# Get optimizers; 1 per source language of encoder and 1 for discriminator
optimizers = {'gen': {}}
for lang in languages['src']:
    optimizers['gen'][lang] = torch.optim.Adam([{'params': net.generator.encoders[lang].parameters()},
                                                {'params': net.generator.decoder.parameters()}
                                               ],
                                               lr=0.001, betas=(0.9, 0.999), eps=1e-08, 
                                               weight_decay=0.01, amsgrad=False)
optimizers['dis'] = torch.optim.Adam(net.discriminator.parameters(),
                                     lr=0.001, betas=(0.9, 0.999), eps=1e-08, 
                                     weight_decay=0.001, amsgrad=False)

### AUTO ENCODER ###
ae = AutoEncoder(embedding_dim, internal_dim, net.generator.decoder)
optimizer_ae = torch.optim.Adam(ae.parameters(),
                                lr=0.01, betas=(0.9, 0.999), eps=1e-08, 
                                weight_decay=0., amsgrad=False)

### NOISY AUTO ENCODER ###
nae = NoisyAutoEncoder(net.generator.encoders['de'], embedding_dim, internal_dim, embedding_dim)
optimizer_nae = torch.optim.Adam(nae.parameters(),
                                 lr=0.01, betas=(0.9, 0.999), eps=1e-08, 
                                 weight_decay=0., amsgrad=False)



# Create toy train dataset
rows, cols = batch_size, embedding_dim
train_data = torch.rand(rows, cols)


# Report outcome before training
reproduction = net.generator(train_data, 'de')
print('Input data:\n', train_data)
print('Reproduced data:\n', reproduction)
print('MSE loss on reproducing input:', AutoEncoderLoss(reproduction, train_data))
print('Start training.')


# Train
for epoch in range(5000):
    
    ### GAN STEP ###
    
    # Update discriminator #
    
    net.discriminator.train()
    net.generator.eval()
    net.discriminator.zero_grad()

    # Labels
    y_true = torch.zeros([nr_langs*batch_size])
    y_true[0:batch_size] = real_label  # First elements are target embeddings


    # Predictions all-real minibatch
    y_preds = torch.zeros([nr_langs*batch_size, 1]) 
    y_preds[0:batch_size] = net.discriminator(train_data)

    # Predictions all-fake minibatch
    x_trans = net.generator(train_data, 'de')  # Generate fake data aka translate
    y_preds[batch_size:] = net.discriminator(x_trans)
    loss_dis = criterion(y_preds, y_true)
    loss_dis.backward()    # Compute gradients only for discriminator
    optimizers['dis'].step() 
    
    # Update generator #
    
    net.generator.train()
    net.discriminator.eval()
    net.generator.zero_grad()
    
    # Labels
    y_true = torch.full((batch_size,), real_label)
    
    # Predictions all-fake minibatch
    x_trans = net.generator(train_data, 'de')
    y_pred = net.discriminator(x_trans)
    loss_gen = criterion(y_pred, y_true)
    loss_gen.backward()    # Compute gradients only for generator
    optimizers['gen']['de'].step()
    
    
    ### AUTO ENCODER STEP ###

    # Perform training
    ae.train()
    optimizer_ae.zero_grad()
    y_pred = ae(train_data)
    loss_ae = AutoEncoderLoss(y_pred, train_data)
    loss_ae.backward()
    optimizer_ae.step()


    ### NOISY AUTO ENCODER STEP ###

    # Perform training
    nae.train()
    nae.zero_grad()
    y_pred = nae(train_data)
    loss_nae = AutoEncoderLoss(y_pred, train_data)
    loss_nae.backward()
    optimizer_nae.step()
    
    
    print('Epoch', epoch, 'Generator loss:', loss_gen.detach().numpy(), 
          'Discriminator loss:', loss_dis.detach().numpy(),
          'AutoEncoder loss:', loss_ae.detach().numpy(),
          'NoisyAutoEncoder loss:', loss_nae.detach().numpy())
    
# Report outcome after training
reproduction = net.generator(train_data, 'de')
print('Training finished.')
print('Input data:\n', train_data)
print('Reproduced data:\n', reproduction)
print('MSE loss on reproducing input:', AutoEncoderLoss(reproduction, train_data))

Input data:
 tensor([[0.2674, 0.0225, 0.5170, 0.1195, 0.7214, 0.9715, 0.2708, 0.2184],
        [0.0835, 0.3893, 0.8359, 0.6106, 0.1536, 0.6743, 0.7822, 0.0313],
        [0.9910, 0.0403, 0.7210, 0.1714, 0.2574, 0.1849, 0.5246, 0.1221],
        [0.9954, 0.4016, 0.3732, 0.4869, 0.2241, 0.6656, 0.0382, 0.0853],
        [0.7299, 0.9797, 0.5991, 0.3265, 0.9785, 0.3437, 0.7733, 0.3440],
        [0.3545, 0.2383, 0.9753, 0.5937, 0.1318, 0.9189, 0.1770, 0.3785],
        [0.5987, 0.8169, 0.6702, 0.2606, 0.5309, 0.2165, 0.3928, 0.9153],
        [0.4063, 0.1753, 0.4112, 0.2999, 0.0733, 0.2434, 0.1043, 0.0789],
        [0.5738, 0.5856, 0.9630, 0.3511, 0.0994, 0.1360, 0.0196, 0.4194],
        [0.8691, 0.0140, 0.0305, 0.3763, 0.5214, 0.0538, 0.3645, 0.3653]])
Reproduced data:
 tensor([[ 0.0708,  0.1947,  0.0756, -0.1646, -0.3265, -0.1186, -0.1922,  0.3298],
        [ 0.0842,  0.1850,  0.0680, -0.1750, -0.3354, -0.0990, -0.2051,  0.3135],
        [ 0.0727,  0.2013,  0.0751, -0.1537, -0.3378, -0.1003, -

Epoch 53 Generator loss: 1.5656569 Discriminator loss: 0.2628836 AutoEncoder loss: 0.056651812 NoisyAutoEncoder loss: 0.042904824
Epoch 54 Generator loss: 1.5726109 Discriminator loss: 0.25301602 AutoEncoder loss: 0.056311168 NoisyAutoEncoder loss: 0.041716818
Epoch 55 Generator loss: 1.5926187 Discriminator loss: 0.24360609 AutoEncoder loss: 0.0559641 NoisyAutoEncoder loss: 0.04082895
Epoch 56 Generator loss: 1.6161762 Discriminator loss: 0.23562758 AutoEncoder loss: 0.05562489 NoisyAutoEncoder loss: 0.039922327
Epoch 57 Generator loss: 1.6445 Discriminator loss: 0.22895405 AutoEncoder loss: 0.055302788 NoisyAutoEncoder loss: 0.039593644
Epoch 58 Generator loss: 1.6764568 Discriminator loss: 0.2234821 AutoEncoder loss: 0.055000417 NoisyAutoEncoder loss: 0.039427318
Epoch 59 Generator loss: 1.7172678 Discriminator loss: 0.21829312 AutoEncoder loss: 0.054745305 NoisyAutoEncoder loss: 0.039710693
Epoch 60 Generator loss: 1.7714102 Discriminator loss: 0.21259058 AutoEncoder loss: 0.054459

Epoch 136 Generator loss: 1.4930437 Discriminator loss: 0.3606356 AutoEncoder loss: 0.027326455 NoisyAutoEncoder loss: 0.0319555
Epoch 137 Generator loss: 1.528683 Discriminator loss: 0.34918746 AutoEncoder loss: 0.02714109 NoisyAutoEncoder loss: 0.03169215
Epoch 138 Generator loss: 1.5595992 Discriminator loss: 0.33859688 AutoEncoder loss: 0.026945243 NoisyAutoEncoder loss: 0.03156983
Epoch 139 Generator loss: 1.5818783 Discriminator loss: 0.32954141 AutoEncoder loss: 0.026766915 NoisyAutoEncoder loss: 0.03063015
Epoch 140 Generator loss: 1.595835 Discriminator loss: 0.32220703 AutoEncoder loss: 0.0266051 NoisyAutoEncoder loss: 0.031904466
Epoch 141 Generator loss: 1.5960648 Discriminator loss: 0.31765133 AutoEncoder loss: 0.026434641 NoisyAutoEncoder loss: 0.030588826
Epoch 142 Generator loss: 1.5886439 Discriminator loss: 0.3153228 AutoEncoder loss: 0.02625699 NoisyAutoEncoder loss: 0.031343646
Epoch 143 Generator loss: 1.6025636 Discriminator loss: 0.30877507 AutoEncoder loss: 0.02

Epoch 212 Generator loss: 1.5005711 Discriminator loss: 0.36987028 AutoEncoder loss: 0.019426366 NoisyAutoEncoder loss: 0.021303106
Epoch 213 Generator loss: 1.4875538 Discriminator loss: 0.37243554 AutoEncoder loss: 0.019297171 NoisyAutoEncoder loss: 0.02171708
Epoch 214 Generator loss: 1.4458274 Discriminator loss: 0.37844485 AutoEncoder loss: 0.019165445 NoisyAutoEncoder loss: 0.02020538
Epoch 215 Generator loss: 1.4010298 Discriminator loss: 0.3870632 AutoEncoder loss: 0.019028004 NoisyAutoEncoder loss: 0.020989686
Epoch 216 Generator loss: 1.3861221 Discriminator loss: 0.3934639 AutoEncoder loss: 0.018884804 NoisyAutoEncoder loss: 0.020995175
Epoch 217 Generator loss: 1.4013773 Discriminator loss: 0.3972804 AutoEncoder loss: 0.018739227 NoisyAutoEncoder loss: 0.021183217
Epoch 218 Generator loss: 1.4442108 Discriminator loss: 0.39726844 AutoEncoder loss: 0.018586058 NoisyAutoEncoder loss: 0.0210996
Epoch 219 Generator loss: 1.5006454 Discriminator loss: 0.39339256 AutoEncoder loss

Epoch 288 Generator loss: 1.5959678 Discriminator loss: 0.44862217 AutoEncoder loss: 0.0108710285 NoisyAutoEncoder loss: 0.015958065
Epoch 289 Generator loss: 1.6216081 Discriminator loss: 0.44117904 AutoEncoder loss: 0.010840422 NoisyAutoEncoder loss: 0.015534818
Epoch 290 Generator loss: 1.6618935 Discriminator loss: 0.4312982 AutoEncoder loss: 0.010810177 NoisyAutoEncoder loss: 0.015845688
Epoch 291 Generator loss: 1.6991904 Discriminator loss: 0.4217915 AutoEncoder loss: 0.010781465 NoisyAutoEncoder loss: 0.01617049
Epoch 292 Generator loss: 1.7303305 Discriminator loss: 0.4138472 AutoEncoder loss: 0.010752402 NoisyAutoEncoder loss: 0.0150248725
Epoch 293 Generator loss: 1.7736565 Discriminator loss: 0.40063435 AutoEncoder loss: 0.010723678 NoisyAutoEncoder loss: 0.015635913
Epoch 294 Generator loss: 1.8189958 Discriminator loss: 0.38596427 AutoEncoder loss: 0.010696484 NoisyAutoEncoder loss: 0.016144836
Epoch 295 Generator loss: 1.852795 Discriminator loss: 0.37270856 AutoEncoder 

...

Epoch 4639 Generator loss: 1.928389 Discriminator loss: 0.26069015 AutoEncoder loss: 5.535296e-05 NoisyAutoEncoder loss: 0.0024025806
Epoch 4640 Generator loss: 1.9823679 Discriminator loss: 0.25188878 AutoEncoder loss: 7.753203e-05 NoisyAutoEncoder loss: 0.0028837633
Epoch 4641 Generator loss: 2.0655403 Discriminator loss: 0.23786923 AutoEncoder loss: 7.59172e-05 NoisyAutoEncoder loss: 0.0029752345
Epoch 4642 Generator loss: 2.0482383 Discriminator loss: 0.23586431 AutoEncoder loss: 5.6554003e-05 NoisyAutoEncoder loss: 0.0027336935
Epoch 4643 Generator loss: 2.0132632 Discriminator loss: 0.2357533 AutoEncoder loss: 6.413379e-05 NoisyAutoEncoder loss: 0.0021761311
Epoch 4644 Generator loss: 2.01721 Discriminator loss: 0.23095436 AutoEncoder loss: 6.5066415e-05 NoisyAutoEncoder loss: 0.0028262502
Epoch 4645 Generator loss: 1.9975258 Discriminator loss: 0.23391578 AutoEncoder loss: 6.115717e-05 NoisyAutoEncoder loss: 0.0021993578
Epoch 4646 Generator loss: 2.0072808 Discriminator loss: 0

Epoch 4708 Generator loss: 1.9641168 Discriminator loss: 0.25992236 AutoEncoder loss: 4.94765e-05 NoisyAutoEncoder loss: 0.0019500017
Epoch 4709 Generator loss: 1.9606583 Discriminator loss: 0.26214904 AutoEncoder loss: 5.033061e-05 NoisyAutoEncoder loss: 0.0025964424
Epoch 4710 Generator loss: 1.9731823 Discriminator loss: 0.2616801 AutoEncoder loss: 5.3304684e-05 NoisyAutoEncoder loss: 0.0020731944
Epoch 4711 Generator loss: 1.9655291 Discriminator loss: 0.26151168 AutoEncoder loss: 5.474249e-05 NoisyAutoEncoder loss: 0.0021028155
Epoch 4712 Generator loss: 1.9225037 Discriminator loss: 0.26763564 AutoEncoder loss: 5.7188998e-05 NoisyAutoEncoder loss: 0.0022037353
Epoch 4713 Generator loss: 1.8952379 Discriminator loss: 0.2728056 AutoEncoder loss: 5.3839864e-05 NoisyAutoEncoder loss: 0.0022768113
Epoch 4714 Generator loss: 1.8628035 Discriminator loss: 0.28091866 AutoEncoder loss: 5.4438213e-05 NoisyAutoEncoder loss: 0.0025132294
Epoch 4715 Generator loss: 1.8517917 Discriminator los

Epoch 4790 Generator loss: 1.7859497 Discriminator loss: 0.304123 AutoEncoder loss: 5.49137e-05 NoisyAutoEncoder loss: 0.0024731946
Epoch 4791 Generator loss: 1.7874641 Discriminator loss: 0.3022752 AutoEncoder loss: 5.3644944e-05 NoisyAutoEncoder loss: 0.002446759
Epoch 4792 Generator loss: 1.7449452 Discriminator loss: 0.30325645 AutoEncoder loss: 5.203117e-05 NoisyAutoEncoder loss: 0.001949649
Epoch 4793 Generator loss: 1.7123907 Discriminator loss: 0.3010274 AutoEncoder loss: 5.1011528e-05 NoisyAutoEncoder loss: 0.0023607393
Epoch 4794 Generator loss: 1.6811367 Discriminator loss: 0.29961115 AutoEncoder loss: 5.0890514e-05 NoisyAutoEncoder loss: 0.0023884673
Epoch 4795 Generator loss: 1.6760677 Discriminator loss: 0.29996395 AutoEncoder loss: 5.178594e-05 NoisyAutoEncoder loss: 0.002824571
Epoch 4796 Generator loss: 1.7164026 Discriminator loss: 0.2962428 AutoEncoder loss: 5.2003732e-05 NoisyAutoEncoder loss: 0.0023351954
Epoch 4797 Generator loss: 1.7759345 Discriminator loss: 0.2

Epoch 4871 Generator loss: 1.8872255 Discriminator loss: 0.2750713 AutoEncoder loss: 0.00013562258 NoisyAutoEncoder loss: 0.0025034205
Epoch 4872 Generator loss: 1.8537178 Discriminator loss: 0.28161126 AutoEncoder loss: 9.664562e-05 NoisyAutoEncoder loss: 0.0025025157
Epoch 4873 Generator loss: 1.8100468 Discriminator loss: 0.28408542 AutoEncoder loss: 7.631689e-05 NoisyAutoEncoder loss: 0.002322449
Epoch 4874 Generator loss: 1.8338816 Discriminator loss: 0.27747485 AutoEncoder loss: 8.009041e-05 NoisyAutoEncoder loss: 0.0023466493
Epoch 4875 Generator loss: 1.8961865 Discriminator loss: 0.26824492 AutoEncoder loss: 9.4397874e-05 NoisyAutoEncoder loss: 0.0020951775
Epoch 4876 Generator loss: 1.8790216 Discriminator loss: 0.2605342 AutoEncoder loss: 9.499745e-05 NoisyAutoEncoder loss: 0.0025535952
Epoch 4877 Generator loss: 1.8340528 Discriminator loss: 0.26312563 AutoEncoder loss: 7.60711e-05 NoisyAutoEncoder loss: 0.0029251429
Epoch 4878 Generator loss: 1.8129715 Discriminator loss: 

Epoch 4947 Generator loss: 1.8006144 Discriminator loss: 0.28447253 AutoEncoder loss: 4.901638e-05 NoisyAutoEncoder loss: 0.0029120627
Epoch 4948 Generator loss: 1.8412241 Discriminator loss: 0.28074273 AutoEncoder loss: 4.9151335e-05 NoisyAutoEncoder loss: 0.002432892
Epoch 4949 Generator loss: 1.896917 Discriminator loss: 0.2753805 AutoEncoder loss: 4.8981827e-05 NoisyAutoEncoder loss: 0.0024349592
Epoch 4950 Generator loss: 1.9423592 Discriminator loss: 0.26988143 AutoEncoder loss: 4.896628e-05 NoisyAutoEncoder loss: 0.0024313666
Epoch 4951 Generator loss: 1.962689 Discriminator loss: 0.26596704 AutoEncoder loss: 4.899013e-05 NoisyAutoEncoder loss: 0.0022099705
Epoch 4952 Generator loss: 1.9627212 Discriminator loss: 0.26303262 AutoEncoder loss: 4.8876835e-05 NoisyAutoEncoder loss: 0.002309566
Epoch 4953 Generator loss: 1.932827 Discriminator loss: 0.26285604 AutoEncoder loss: 4.908422e-05 NoisyAutoEncoder loss: 0.0022365195
Epoch 4954 Generator loss: 1.9009607 Discriminator loss: 0